--- 

# 2. Dataset Preparation
---
For this Capstone, are eelected to be procesed two files from #[CSE-CIC-IDS2018](https://www.unb.ca/cic/datasets/ids-2018.html) those are 
- `Friday-16-02-2018_TrafficForML_CICFlowMeter.csv`
This file contains most of Dos attacks

- `Friday-02-03-2018_TrafficForML_CICFlowMeter.csv`
This file contains most of botnet computers.

since these two files contains a large malicius packages, it will help help to balance the dataset which will be uses to train the model.


In [ ]:
%%time
import os
import pandas as pd
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Loading path to dataset files.

In [ ]:
DATASET_FILES_PATH = []
for path, _, file in (os.walk("./datasets/")):
    for eachFile in file:
        DATASET_FILES_PATH.append(path + eachFile)
DATASET_FILES_PATH

## Data Ingestion / versioning
### Concatenating datasetsLoading datasets to PandaData Frame

In [ ]:
%%time

# df_dataset = pd.read_csv(DATASET_FILES_PATH[0])
# print(df_dataset.shape)

# For Google Colab, due to memory capacity, only can handle one day dataset.
df_friday1 = pd.read_csv(DATASET_FILES_PATH[0])
df_friday2 = pd.read_csv(DATASET_FILES_PATH[1])
# # For Google Colab, due to memory capacity, only can handle one day dataset.
df_dataset = pd.concat([df_friday1, df_friday2], axis=0, ignore_index=True)
# # Because two datasets was concatenated, then need to delete the row which cointain the second dataframe title
df_dataset.drop(df_dataset.loc[df_dataset["Label"] == "Label"].index, inplace=True)
print(df_dataset.shape)

In [ ]:
################### TEMP

# # para agilizar y progar

# # Drop rows from index 30 to 100
# df_dataset = df_dataset.drop(index=range(200000, 1048575))
# print(df_dataset.shape)


##TEMMPPP

## Data Cleaning
- ### Drop unrelated columns
Since Port, protocol and the timestand are not related to the label with those selectec machine learning, those will be droped

In [ ]:
df_dataset.drop(columns=['Dst Port', 'Protocol', 'Timestamp'], inplace=True)



- ### Droping rows with infinite or null values

In [ ]:
print("Shape before deleting rows: ", df_dataset.shape)
df_dataset[df_dataset.isnull().any(axis=1)]
df_dataset.replace([np.inf, -np.inf], np.nan)
df_dataset.dropna(inplace=True)
print("Shape after deteling rows:", df_dataset.shape)

## Encoding
### Check Label labels

In [ ]:
print(df_dataset['Label'].unique())
print(df_dataset.shape)

##### Changing Labels names 
To unify the labels, those malicius packages will be renamend as ones, and the normal as zeros.
- 0 - normal package
- 1 - malicius package

In [ ]:
%%time
df_dataset.replace(to_replace=['Benign'], value=0, inplace=True)
df_dataset.replace(to_replace=["Bot", "DoS attacks-SlowHTTPTest", "DoS attacks-Hulk"], value=1, inplace=True)
df_dataset[df_dataset.columns[-1]].unique()
# some values are saved as string, but actually they should be integer values, forcing here changing types
df_dataset.astype('float')

### Dropping duplicated rows

In [ ]:
print(df_dataset.shape)
df_dataset.drop_duplicates(inplace=True)
print(df_dataset.shape)


### Check columns datatypes

In [ ]:
df_dataset.info()

### Distributions labels after drop rows

In [ ]:
label_benign = df_dataset["Label"].value_counts()[[0]].sum()
label_malicious = df_dataset["Label"].value_counts()[[1]].sum()

print(df_dataset.shape)

abs_values = [label_benign, label_malicious]
sns.set(rc={'figure.figsize':(8, 6)})
ax = sns.countplot(x=df_dataset[df_dataset.columns[-1]], 
              data = df_dataset,
              palette = 'dark:#5A9_r')
ax.bar_label(container=ax.containers[0], labels=[label_benign])
# ax.bar_label(container=ax.containers[1], labels=[label_malicious]) # fails compile in google colab
plt.xlabel(f"0 = Bening; 1 = Malicious")


##### Inbalance problem
There are two ways to solve this problem, 
1. Droping bening rows.
2. Sampling Malicious rows.

for this attempt, I am dropping bening rows until it gets balanced.


In [ ]:
df_dataset.drop(df_dataset[df_dataset.Label == 0].index[-(label_benign-label_malicious):], inplace=True)

### Distributions labels after drop fixing imbalance problem

In [ ]:
label_benign = df_dataset["Label"].value_counts()[[0]].sum()
label_malicious = df_dataset["Label"].value_counts()[[1]].sum()

print(df_dataset.shape)

abs_values = [label_benign, label_malicious]
sns.set(rc={'figure.figsize':(8, 6)})
ax = sns.countplot(x=df_dataset[df_dataset.columns[-1]], 
              data = df_dataset,
              palette = 'dark:#5A9_r')
ax.bar_label(container=ax.containers[0], labels=[label_benign])
# ax.bar_label(container=ax.containers[1], labels=[label_malicious]) # fails compile on google colab
plt.xlabel(f"0 = Bening; 1 = Malicious")

### Saving the Dataset as a csv file

In [ ]:
df_dataset.to_csv("processed_dataset_in_2.csv", index=False)